In [5]:
import copy
import itertools
import functools
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as torch_transforms
import encoding.datasets as enc_ds
import torchvision.transforms as transforms

In [4]:
encoding_datasets = {
    x: functools.partial(enc_ds.get_dataset, x)
    for x in ["coco", "ade20k", "pascal_voc", "pascal_aug", "pcontext", "citys"]
}
encoding_datasets

{'coco': functools.partial(<function get_dataset at 0x7f9a90b29c10>, 'coco'),
 'ade20k': functools.partial(<function get_dataset at 0x7f9a90b29c10>, 'ade20k'),
 'pascal_voc': functools.partial(<function get_dataset at 0x7f9a90b29c10>, 'pascal_voc'),
 'pascal_aug': functools.partial(<function get_dataset at 0x7f9a90b29c10>, 'pascal_aug'),
 'pcontext': functools.partial(<function get_dataset at 0x7f9a90b29c10>, 'pcontext'),
 'citys': functools.partial(<function get_dataset at 0x7f9a90b29c10>, 'citys')}

In [7]:
norm_mean= [0.5, 0.5, 0.5]
norm_std = [0.5, 0.5, 0.5]
train_transform = [
            transforms.ToTensor(),
            transforms.Normalize(norm_mean, norm_std),
        ]
train_transform = transforms.Compose(train_transform)
kwargs = {'root': '../datasets/', 
          'split': 'train', 
          'mode': 'train', 
          'transform': train_transform, 
          'base_size': 520,
          'crop_size': 480}
name = 'ade20k'
dataset = encoding_datasets[name.lower()](**kwargs)

BaseDataset: base_size 520, crop_size 480
len(img_paths): 20210


In [11]:
dataset[0][1]

tensor([[  0,   0,   0,  ...,  -1,  -1,  -1],
        [  0, 134, 134,  ...,  -1,  -1,  -1],
        [  0, 134, 134,  ...,  -1,  -1,  -1],
        ...,
        [ -1,  -1,  -1,  ...,  -1,  -1,  -1],
        [ -1,  -1,  -1,  ...,  -1,  -1,  -1],
        [ -1,  -1,  -1,  ...,  -1,  -1,  -1]])